In [1]:
!pip install psycopg2-binary boto3 sqlalchemy


In [4]:
!pip install supabase

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/14 [pydantic-core]
      Successfully uninstalled pydantic_core-2.23.4━━━━━━━━━━━  3/14 [pydantic-core]
  Attempting uninstall: pydantic

In [13]:

# print("Supabase raw response:", response)
# print("Supabase data:", response.data)


# print("DataFrame shape:", df.shape)
# print("DataFrame head:\n", df.head())


from supabase import create_client, Client
import pandas as pd
import boto3

# Supabase config
SUPABASE_URL = "https://mcgzvjzuqnjstptfrsuj.supabase.co"
SUPABASE_KEY = "sb_publishable_4vx6B3HjwnEQVwI9SgrJkQ_S32tu5l5"

# Use the service_role key, not anon

# Create Supabase client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Fetch data
response = supabase.table("loan_fraud_analytics").select("*").execute()
df = pd.DataFrame(response.data)

# Save as CSV
csv_path = "/tmp/loan_fraud_data.csv"
df.to_csv(csv_path, index=False)

# Upload to S3
s3 = boto3.client("s3")
bucket_name = "manas-bucket100"
object_key = "input-file/loan_fraud_data.csv"

s3.upload_file(csv_path, bucket_name, object_key)
print("✅ Upload to S3 complete")


✅ Upload to S3 complete


In [15]:
import boto3
import io

# Drop target column
df_features = df.drop(columns=['loan_default'])

# Convert to CSV buffer
csv_buffer = io.StringIO()
df_features.to_csv(csv_buffer, index=False)

# Upload to S3
s3 = boto3.client('s3')
bucket_name = 'manas-bucket100'  # 🔁 Your bucket name
object_key = 'input-file/loan_fraud_data.csv'

s3.put_object(Bucket=bucket_name, Key=object_key, Body=csv_buffer.getvalue())

print(f"✅ Features uploaded to s3://{bucket_name}/{object_key}")


✅ Features uploaded to s3://manas-bucket100/input-file/loan_fraud_data.csv


In [16]:
####### Code for Data processing ########
import pandas as pd
import numpy as np
import os
import boto3
from io import StringIO
bucket_name = "manas-bucket100"
s3_key = "input-file/loan_fraud_data.csv"

# Step 1: Define S3 download
def load_csv_from_s3(bucket_name, file_key):
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    content = response['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(content))
    print(f"✅ Loaded data from s3://{bucket_name}/{file_key}")
    return df

# Step 2: Handle missing values
def handle_missing_values(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)
    return df

# Step 3: Handle outliers
def handle_outliers(df):
    df = df.copy()
    for col in df.select_dtypes(include=np.number).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df[col] = np.where(df[col] < lower, lower,
                  np.where(df[col] > upper, upper, df[col]))
    return df

# Step 4: Create dummies
def create_dummies(df):
    df = pd.get_dummies(df, drop_first=True, dtype=int)
    return df

# Step 5: Drop low-variance and ID-like columns
def drop_low_variance_and_id_columns(df, threshold=0.95):
    df = df.copy()
    drop_cols = []

    for col in df.columns:
        if df[col].nunique() <= 1:
            drop_cols.append(col)
        else:
            top_freq_ratio = df[col].value_counts(normalize=True).values[0]
            if top_freq_ratio >= threshold:
                drop_cols.append(col)

    id_like_cols = [col for col in df.columns if col.lower() == 'id'
                    or col.lower().startswith('id')
                    or col.lower().endswith('id')
                    or '_id' in col.lower()
                    or 'id_' in col.lower()]
    
    drop_cols = list(set(drop_cols + id_like_cols))

    if drop_cols:
        print(f"🧹 Dropping low-variance/id-like columns: {drop_cols}")
        df.drop(columns=drop_cols, inplace=True)

    return df

# Step 6: Run all preprocessing
def preprocess_data(df):
    df = drop_low_variance_and_id_columns(df, threshold=0.95)
    df = handle_missing_values(df)
    df = handle_outliers(df)
    df = create_dummies(df)
    return df

# MAIN execution inside SageMaker
def main():
    bucket_name = 'manas-bucket100'
    file_key = 'input-file/loan_fraud_data.csv'  # full path in S3 bucket

    df = load_csv_from_s3(bucket_name, file_key)
    print(f"📥 Original shape: {df.shape}")

    df = preprocess_data(df)
    print(f"📊 Processed shape: {df.shape}")

    # Save to local file or upload to S3 if needed
    processed_file = "processed_test_data.csv"
    df.to_csv(processed_file, index=False)
    print(f"✅ Preprocessed test data saved: {processed_file}")

    return df

# Run if in script mode
if __name__ == "__main__":
    main()


✅ Loaded data from s3://manas-bucket100/input-file/loan_fraud_data.csv
📥 Original shape: (1000, 9)
🧹 Dropping low-variance/id-like columns: ['loan_id', 'last_delinq_none']
📊 Processed shape: (1000, 10)
✅ Preprocessed test data saved: processed_test_data.csv


/tmp/ipykernel_10936/2229654881.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipykernel_10936/2229654881.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr